<a href="https://colab.research.google.com/github/adalves-ufabc/2024.Q2-PLN/blob/main/2024_Q2_PLN_AULA_11_Notebook_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2024-Q2]**
Prof. Alexandre Donizeti Alves

## **LangChain**
---



In [ ]:
#@title Instalando o pacote LangChain
!pip install langchain -q U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.6/377.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.2 MB/s eta 0:00:00


In [ ]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

0.2.11


In [ ]:
#@title Integração com o pacote da OpenAI

!pip install -qU langchain-openai

In [ ]:
#@title Definindo a chave da API da OpenAI

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## **Cadeia (*Chain*)**

No **LangChain**, uma **cadeia** é uma sequência de operações que conecta vários componentes, como modelos de linguagem, *parsers* de saída (um componente que processa a saída do modelo para formatação) e transformações de dados, para criar um fluxo de trabalho coeso. A **cadeia** permite combinar diferentes passos de processamento e manipulação de dados de maneira estruturada e eficiente.

As **cadeias** são configuradas para permitir um fluxo contínuo de dados entre os componentes, facilitando a construção de aplicações complexas de PLN.

Podemos então inicializar o modelo:

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI( temperature = 0.9, max_tokens= 50 )

Depois de instalar e inicializar o modelo de sua escolha, podemos tentar usá-lo!

In [ ]:
prompt = "Qual seria um bom nome para uma empresa que fabrica meias coloridas? Responda em Português e retorne apenas o nome da empresa"

modelo.invoke(prompt)

AIMessage(content='ColorMeias', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 38, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4daeb7f0-04ea-40d1-ba46-a3465444c341-0', usage_metadata={'input_tokens': 38, 'output_tokens': 3, 'total_tokens': 41})

**Templates de *prompt***

Também podemos orientar a resposta com um template de *prompt*. Os templates de *prompt* são usados para converter a entrada bruta do usuário em uma entrada melhor para o modelo.

**`PromptTemplate`**

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables = ["produto"],
    template = "Qual seria um bom nome para uma empresa que fabrica {produto}? Responda em Português e retorne apenas o nome da empresa?",
)

Agora podemos combiná-los em uma cadeia simples:

In [ ]:
chain = prompt | modelo

In [ ]:
chain.invoke({"produto": "meias coloridas"})

AIMessage(content='Arco-Íris Meias Coloridas.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 39, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-91b7c37b-9330-49d7-8069-3d27ade1fb4e-0', usage_metadata={'input_tokens': 39, 'output_tokens': 10, 'total_tokens': 49})

**`ChatPromptTemplate`**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um estrategista de marketing digital especializado em {setor}"),
    ("user", "Considerando a crescente digitalização no {setor}, proponha um slogan cativante para uma campanha publicitária.")
])

chain = prompt | modelo

In [ ]:
chain.invoke({"setor": "contábil"})

AIMessage(content='"Transforme sua contabilidade, conecte-se ao futuro!"', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 55, 'total_tokens': 68}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d9ae5d3d-9f66-425a-aa55-14f34f447c39-0', usage_metadata={'input_tokens': 55, 'output_tokens': 13, 'total_tokens': 68})

**`MessagesPlaceholder`**

Este modelo de *prompt* é responsável por adicionar uma lista de mensagens em um determinado local.

 `MessagesPlaceholder` é uma classe usada no **LangChain** para representar um espaço reservado dentro de um modelo de *prompt*, onde mensagens dinâmicas podem ser inseridas posteriormente. Ele atua como um marcador que indica onde as mensagens devem ser incluídas na estrutura do *prompt*.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# Define o template de prompt com uma mensagem do sistema e um espaço reservado para mensagens do usuário
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente de viagem experiente. Ajude o usuário a planejar uma viagem."),
    MessagesPlaceholder("usuario_msgs")
])

# Cria um dicionário com mensagens do usuário
mensagens_usuario = [
    HumanMessage(content="Estou planejando uma viagem para a Europa. Quais são as melhores cidades para visitar?"),
    HumanMessage(content="Pode me dar sugestões de atividades em Paris?")
]

# Invoca o template com as mensagens do usuário
resultado = prompt_template.invoke({"usuario_msgs": mensagens_usuario})

In [ ]:
resultado

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente de viagem experiente. Ajude o usuário a planejar uma viagem.'), HumanMessage(content='Estou planejando uma viagem para a Europa. Quais são as melhores cidades para visitar?'), HumanMessage(content='Pode me dar sugestões de atividades em Paris?')])

In [ ]:
resultado.messages[0].content

'Você é um assistente de viagem experiente. Ajude o usuário a planejar uma viagem.'

Integrando com um modelo de linguagem:

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI( temperature = 0.9, max_tokens= 512 )

In [ ]:
chain = prompt_template | modelo

In [ ]:
resposta = chain.invoke({"usuario_msgs": mensagens_usuario})

In [ ]:
from IPython.display import Markdown

Markdown(resposta.content)

Claro! Paris é uma cidade incrível com muitas opções de atividades. Aqui estão algumas sugestões do que fazer na Cidade Luz:

1. Visite a Torre Eiffel: Suba até o topo para ter uma vista deslumbrante da cidade.
2. Passeie pelo Louvre: Explore um dos maiores e mais famosos museus do mundo, lar de obras-primas como a Mona Lisa.
3. Caminhe ao longo do rio Sena: Aproveite a beleza da cidade enquanto passeia pelas margens do rio.
4. Conheça a Catedral de Notre-Dame: Visite essa icônica catedral gótica, que é um marco histórico de Paris.
5. Faça um passeio de barco pelo Sena: Desfrute de uma vista panorâmica da cidade a partir da água.
6. Visite o Palácio de Versalhes: Faça uma viagem de um dia para conhecer esse magnífico palácio e seus belos jardins.
7. Explore o bairro de Montmartre: Descubra a atmosfera boêmia desse bairro, lar do famoso Moulin Rouge e da Basílica de Sacré-Cœur.
8. Delicie-se com a gastronomia francesa: Experimente os deliciosos queijos, vinhos, croissants e outras iguarias locais.

Essas são apenas algumas das muitas atividades que Paris tem a oferecer. Aproveite sua viagem!

**Referência**:

> https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel




## **LangChain Expression Language (LCEL)**

A ***LangChain Expression Language*** é uma forma de criar cadeias personalizadas e arbitrárias. Ela é construída sobre o protocolo `Runnable`.

O protocolo `Runnable` do **LangChain** é uma interface que define como diferentes componentes podem ser encadeados e executados dentro de um fluxo de trabalho.

**Como encadear *runnables***

Um ponto sobre a **LCEL** é que qualquer dois *runnables* podem ser 'encadeados' juntos em sequências. A saída da chamada `.invoke()` do *runnable* anterior é passada como entrada para o próximo *runnable*. Isso pode ser feito usando o operador pipe (`|`), ou o método mais explícito `.pipe()`, que faz a mesma coisa.

A *RunnableSequence* resultante é em si mesma um *runnable*, o que significa que pode ser invocada, transmitida ou encadeada ainda mais, assim como qualquer outro *runnable*.

Para demonstrar como isso funciona, vamos apresentar um exemplo:

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("me conte uma piada sobre {assunto}")

chain = prompt | modelo | StrOutputParser()

In [ ]:
chain.invoke({"assunto": "papagaio"})

'Por que o papagaio foi expulso da escola?\n\nPorque ele só sabia repetir de ano em ano!'

**Coerção**

Podemos até combinar essa cadeia com mais *runnables* para criar outra cadeia. Isso pode envolver algum formato de entrada/saída usando outros tipos de *runnables*, dependendo dos requisitos de entrada e saída dos componentes da cadeia.

Por exemplo, digamos que queremos compor a cadeia de geração de piadas com outra cadeia que avalia se a piada gerada foi engraçada ou não.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

prompt_analise = ChatPromptTemplate.from_template("essa piada é engraçada? {piada}")

chain_composto = {"piada": chain} | prompt_analise | modelo | StrOutputParser()

chain_composto.invoke({"assunto": "papagaio"})

'Sim, essa piada é engraçada porque faz um trocadilho com o fato de que papagaios são conhecidos por repetir o que ouvem, e no caso do advogado, ele repetiria o que o cliente diz durante o processo.'

No contexto do **LangChain**, **coerção** refere-se ao processo de converter automaticamente ou adaptar um tipo de dado para outro, de modo que ele se encaixe nos requisitos de entrada ou saída de uma cadeia ou componente específico. Isso pode acontecer, por exemplo, quando você passa dados entre diferentes *runnables* (componentes que seguem o protocolo `Runnable`) em uma cadeia, e o **LangChain** ajusta automaticamente os dados para que sejam compatíveis com o próximo componente.

Por exemplo, se você passar um dicionário como entrada em uma cadeia que espera um formato específico, o **LangChain** pode coagir (converter) automaticamente esse dicionário para o formato adequado, como um `RunnableParallel`, que executa operações em paralelo.

Esse mecanismo de coerção facilita a construção de cadeias complexas, pois minimiza a necessidade de manipulação manual dos dados para garantir que sejam compatíveis com cada etapa do processo.

**O método `.pipe()`**

In [ ]:
from langchain_core.runnables import RunnableParallel

chain_composto_pipe = (
    RunnableParallel({"piada": chain})
    .pipe(prompt_analise)
    .pipe(modelo)
    .pipe(StrOutputParser())
)

chain_composto_pipe.invoke({"assunto": "papagaio"})

'Sim, essa piada é engraçada porque faz uma brincadeira com o fato de que os papagaios costumam repetir o que ouvem. Eles não gostam de piadas porque acham que são sempre as mesmas e previsíveis.'

Ou de forma abreviada:

In [ ]:
chain_composto_pipe = RunnableParallel({"piada": chain}).pipe(
    prompt_analise, modelo, StrOutputParser()
)

chain_composto_pipe.invoke({"assunto": "papagaio"})

'Sim, essa piada é engraçada porque brinca com a ideia de um papagaio que fala demais e acaba sendo expulso da escola por isso. A situação inusitada e o trocadilho com o fato do papagaio falar "besteira" durante as aulas contribuem para o humor da piada.'

**Função de Depuração**

Uma função de depuração é uma função usada para inspecionar, verificar ou diagnosticar o estado de um programa durante sua execução.

In [ ]:
from langchain_core.runnables import RunnableParallel
from langchain_core.messages import HumanMessage

def funcao_debug(output):
    print("Piada gerada:", output["piada"])
    return output

chain_composto_pipe = (
    RunnableParallel({"piada": chain})
    .pipe(funcao_debug)  # adiciona uma função de depuração
    .pipe(prompt_analise)
    .pipe(modelo)
    .pipe(StrOutputParser())
)

chain_composto_pipe.invoke({"assunto": "papagaio"})

Piada gerada: Por que o papagaio não consegue segurar um segredo? Porque ele sempre vai acabar papagaiando!


'Sim, é engraçada porque brinca com o fato de que os papagaios são conhecidos por repetir palavras e segredos, então é impossível para eles manterem algo em segredo.'

**Paralelizar etapas**

`RunnableParallels` facilitam a execução de múltiplos *runnables* em paralelo e o retorno da saída desses *runnables* como um mapa.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI()

chain_piada = ChatPromptTemplate.from_template("me conte uma piada sobre {assunto}") | modelo
chain_poema = ChatPromptTemplate.from_template("escreva um poema de 2 linhas sobre {assunto}") | modelo

chain_mapa = RunnableParallel(piada = chain_piada, poema = chain_poema)

resposta = chain_mapa.invoke({"assunto": "papagaio"})

In [ ]:
resposta

{'piada': AIMessage(content='Por que o papagaio não entra no barco? Porque ele tem medo de se afogar em tanto "papagaio"!', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 16, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e70af190-be80-49d9-9566-b0082d8f681a-0', usage_metadata={'input_tokens': 16, 'output_tokens': 32, 'total_tokens': 48}),
 'poema': AIMessage(content='Papagaio colorido, voa no céu brilhante\nCom seu canto alegre, encanta toda a gente.', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 22, 'total_tokens': 53}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-34b0aff1-e80f-41b0-a6c1-5cbddc64025e-0', usage_metadata={'input_tokens': 22, 'output_tokens': 31, 'total_tokens': 53})}

In [ ]:
resposta.keys()

dict_keys(['piada', 'poema'])

In [ ]:
resposta["piada"]

AIMessage(content='Por que o papagaio não entra no barco? Porque ele tem medo de se afogar em tanto "papagaio"!', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 16, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e70af190-be80-49d9-9566-b0082d8f681a-0', usage_metadata={'input_tokens': 16, 'output_tokens': 32, 'total_tokens': 48})

In [ ]:
resposta["piada"].content

'Por que o papagaio não entra no barco? Porque ele tem medo de se afogar em tanto "papagaio"!'

`RunnableParallel` também é útil para executar processos independentes em paralelo, uma vez que cada *Runnable* no mapa é executado em paralelo. Por exemplo, podemos ver que nossas cadeias anteriores `chain_piada`, `chain_poema` e `chain_mapa` têm tempos de execução semelhantes, mesmo que `chain_mapa` execute as duas outras cadeias.

In [ ]:
%%timeit

chain_piada.invoke({"assunto": "papagaio"})

960 ms ± 74.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

chain_poema.invoke({"assunto": "papagaio"})

888 ms ± 285 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

chain_mapa.invoke({"assunto": "papagaio"})

996 ms ± 166 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
